# DATA

In [1]:
import os
import random

batch_size = 512

def create_dataset(path):
    dataset = []
    for root, dirs, files in os.walk(path):
        for file in files:
            name = file.split("__")[1]
            if "_" not in name and "f" not in name and "r" not in name:
                file = file.split(".")[0]
                for i in range(10):
                    input1 = root + "/" + file + ".png"
                    input2 = root + "/" + file + "_" + str(i) + ".png"
                    GT = root + "/" + file + "_" + str(i) + "_GT.png"
                    dataset.append((input1, input2, GT))
            elif ("_flip_" in name or "_rot_" in name) and "GT" not in name:
                file = file.split(".")[0]
                if file[-1] in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                    continue
                for i in range(10):
                    input1 = root + "/" + file + ".png"
                    input2 = root + "/" + file + "_" + str(i) + ".png"
                    GT = root + "/" + file + "_" + str(i) + "_GT.png"
                    dataset.append((input1, input2, GT))
    return dataset

train = create_dataset("train")
test = create_dataset("test")
validation = create_dataset("validation")

x_train = []
y_train = []
x_test = []
y_test = []
x_validation = []
y_validation = []

for i in range(0, len(train), 8): #TODO:
    x_train.append((train[i][0], train[i][1]))
    y_train.append(train[i][2])

for i in range(len(test)):
    x_test.append((test[i][0], test[i][1]))
    y_test.append(test[i][2])

for i in range(0, len(validation), 8):
    x_validation.append((validation[i][0], validation[i][1]))
    y_validation.append(validation[i][2])

def get_batch(x, y, batch_size):
    # shuffle x and y
    zipped = list(zip(x, y))
    random.shuffle(zipped)
    x, y = zip(*zipped)
    batch = []
    for i in range(batch_size):
        batch.append((x[i], y[i]))
    return batch

def get_non_random_batch(x, y, batch_size, start_index):
    batch = []
    for i in range(batch_size):
        batch.append((x[start_index + i], y[start_index + i]))
    return batch

# Search functions and planners evaluation

In [3]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import heapq

# Define the heuristic functions
def manhattan_distance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1])

def focal_search(w, grid, start, goal, probability_map, cf, cf_map, eps):
    # Initialize the open and closed lists
    open_list = []
    focal_list = []
    closed_list = set()

    # Add the start node to the open list with the priority equal to the heuristic
    heapq.heappush(open_list, (manhattan_distance(start, goal), 0, start))
    focal_list.append((manhattan_distance(start, goal), probability_map[start[0]][start[1]], start))
    
    came_from = {start: None}
    
    g_costs = {start: 0}
    
    while open_list:
        if focal_list:
            focal_list.sort(key=lambda x: x[1], reverse=True)
            current_f, current_g, current = focal_list.pop(0)
            # remove from open list
            open_list = [node for node in open_list if node[2] != current]
            heapq.heapify(open_list)
        else:
            current_f, current_g, current = heapq.heappop(open_list)
        
        closed_list.add(current)
        
        if current == goal:
            path = []
            while current is not None:
                path.append(current)
                current = came_from[current]
            return path[::-1], closed_list
        
        # Update focal list if necessary
        new_focal_list = []
        f_min = open_list[0][0] if open_list else float('inf')
        for node in open_list:
            if node[0] <= eps * f_min:
                new_focal_list.append((node[0], probability_map[node[2][0]][node[2][1]], node[2]))
        focal_list = new_focal_list
        
        # Check the neighbors of the current node
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            
            # Ensure the neighbor is within the grid bounds and is walkable
            if 0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0]) and grid[neighbor[0]][neighbor[1]] == 1:
                if neighbor in closed_list:
                    continue
                
                # Calculate the tentative g cost
                tentative_g_cost = g_costs[current] + 1
                
                if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                    # Update the cost and path
                    g_costs[neighbor] = tentative_g_cost
                    if cf:
                        if cf_map[neighbor[0]][neighbor[1]] > 0:
                            f_cost = tentative_g_cost + w * manhattan_distance(neighbor, goal) / cf_map[neighbor[0]][neighbor[1]]
                        else:
                            cf_map[neighbor[0]][neighbor[1]] = 0.001
                            f_cost = tentative_g_cost + w * manhattan_distance(neighbor, goal) / cf_map[neighbor[0]][neighbor[1]]
                    else:
                        f_cost = tentative_g_cost + w * manhattan_distance(neighbor, goal)
                    heapq.heappush(open_list, (f_cost, tentative_g_cost, neighbor))
                    if f_cost <= eps * f_min:
                        focal_list.append((f_cost, probability_map[neighbor[0]][neighbor[1]], neighbor))
                    came_from[neighbor] = current
    # If the goal was not reached, return None
    return None, closed_list


In [4]:
def manhattan_distance(node1, node2):
    return abs(node1[0] - node2[0]) + abs(node1[1] - node2[1])

def get_neighbors(node, input_image):
    neighbors = []
    for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_y, new_x = node[1] + dy, node[0] + dx
        if new_y >= 0 and new_y < input_image.shape[1] and new_x >= 0 and new_x < input_image.shape[0] and input_image[new_x, new_y] == 1:
            neighbors.append((new_x, new_y))
    return neighbors

def a_star_GT(w, input_image, start, target):
    # run a_star algorithm from target to start
    shape = input_image.shape
    g_values = {target: 0}
    for y in range(shape[0]):
        for x in range(shape[1]):
            if (y, x) != target:
                g_values[(y, x)] = float('inf')
    open_list = [(target, 0, None)]
    closed = []
    
    while open_list:
        node = open_list[0]
        open_list.remove(node)
        closed.append(node)
        if node[0] == start:
            path = []
            while node is not None:
                path.append(node[0])
                node = node[2]
            path.reverse()
            return path, closed
        
        for neigh in get_neighbors(node[0], input_image):
            if g_values[node[0]] + 1 < g_values[neigh]:
                g_values[neigh] = g_values[node[0]] + 1
                f = g_values[neigh] + w * manhattan_distance(neigh, start)
                if (neigh, f, node) not in closed:
                    open_list.append((neigh, f, node))
    return None, closed

# ici
def a_star_manhattan(w, input_image, start, target):
    import heapq
    
    open = []
    closed = set()
    g_values = {}

    heapq.heappush(open, (0, start, None))
    g_values[start] = 0
    
    img_size = input_image.shape
    
    while open:
        current = heapq.heappop(open)
        current_f, current_node, current_parent = current
        
        closed.add(current_node)
        
        if current_node == target:
            path = []
            while current:
                path.append(current[1])
                current = current[2]
            return path[::-1], closed
        
        for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new_y, new_x = current_node[1] + dy, current_node[0] + dx
            neighbor = (new_x, new_y)
            if (new_y < 0 or new_y >= img_size[1] or new_x < 0 or new_x >= img_size[1] or input_image[neighbor] == 0 or neighbor in closed):
                continue
            
            tentative_g = g_values[current_node] + 1
            if neighbor not in g_values or tentative_g < g_values[neighbor]:
                g_values[neighbor] = tentative_g
                f = tentative_g + w * manhattan_distance(neighbor, target)
                heapq.heappush(open, (f, neighbor, current))
    return None, closed

def a_star(w, input_image, start, target, output):
    import heapq
    def is_valid(x, y):
        return x >= 0 and x < img_size[0] and y >= 0 and y < img_size[1] and input_image[x, y] != 0
    
    def manhattan_distance(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    open = []
    closed = set()
    g_values = {start: 0}
    parent_map = {}

    heapq.heappush(open, (0, start))
    open_set = {start}
    
    img_size = input_image.shape
    
    while open:
        current_f, current_node = heapq.heappop(open)
        open_set.remove(current_node)
        
        closed.add(current_node)
        
        if current_node == target:
            path = []
            while current_node:
                path.append(current_node)
                current_node = parent_map.get(current_node)
            return path[::-1], closed
        
        for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            neighbor = (current_node[0] + dx, current_node[1] + dy)
            if not is_valid(*neighbor) or neighbor in closed:
                continue
            
            tentative_g = g_values[current_node] + 1
            if neighbor not in g_values or tentative_g < g_values[neighbor]:
                g_values[neighbor] = tentative_g
                cf = output[neighbor]
                if cf == 0:
                    cf = 0.01
                f = tentative_g + w * manhattan_distance(neighbor, target) / cf
                parent_map[neighbor] = current_node
                if neighbor not in open_set:
                    heapq.heappush(open, (f, neighbor))
                    open_set.add(neighbor)
    return None, closed

In [5]:
def everything_new(w, input_image, start, target, output_ppm, output_cf, eps):
    import time
    rt_cf = time.time()
    path_pred_cf, visited_cf = a_star(w, input_image, start, target, output_cf)
    rt_cf = time.time() - rt_cf
    rt_ppm_man = time.time()
    path_pred_ppm_man, visited_ppm_man = focal_search(w, input_image, start, target, output_ppm, False, None, eps)
    rt_ppm_man = time.time() - rt_ppm_man
    rt_ppm_cf = time.time()
    path_pred_ppm_cf, visited_ppm_cf = focal_search(w, input_image, start, target, output_ppm, True, output_cf, eps)
    rt_ppm_cf = time.time() - rt_ppm_cf
    path_pred_GT, _ = a_star_GT(1, input_image, start, target)
    rt_man = time.time()
    path_pred_man, visited_man = a_star_manhattan(1, input_image, start, target)
    rt_man = time.time() - rt_man
    
    # test output_cf - output_ppm
    rt_cf_ppm_25 = time.time()
    path_pred_cf_ppm25, visited_cf_ppm25 = a_star(w, input_image, start, target, output_cf + 0.25 * output_ppm)
    rt_cf_ppm_25 = time.time() - rt_cf_ppm_25
    rt_cf_ppm_50 = time.time()
    path_pred_cf_ppm50, visited_cf_ppm50 = a_star(w, input_image, start, target, output_cf + 0.50 * output_ppm)
    rt_cf_ppm_50 = time.time() - rt_cf_ppm_50
    rt_cf_ppm_75 = time.time()
    path_pred_cf_ppm75, visited_cf_ppm75 = a_star(w, input_image, start, target, output_cf + 0.75 * output_ppm)
    rt_cf_ppm_75 = time.time() - rt_cf_ppm_75
    rt_cf_ppm_1 = time.time()
    path_pred_cf_ppm1, visited_cf_ppm1 = a_star(w, input_image, start, target, output_cf + 1 * output_ppm)
    rt_cf_ppm_1 = time.time() - rt_cf_ppm_1
    # if output_ppm < 0.5, make its value to 1
    output_ppm1 = output_ppm.copy()
    output_ppm1[output_ppm1 < 0.75] = 10
    out_cf_sur_ppm = output_cf / output_ppm1
    rt_cf_sur_ppm = time.time()
    path_pred_cf_sur_ppm, visited_cf_sur_ppm = a_star(w, input_image, start, target, out_cf_sur_ppm)
    rt_cf_sur_ppm = time.time() - rt_cf_sur_ppm
    
    """ plt.imshow(output_cf, cmap='gray')
    plt.show()
    
    plt.imshow(output_ppm, cmap='gray')
    plt.show()
    
    plt.imshow(out_cf_sur_ppm, cmap='gray')
    plt.show()
    
    plt.imshow(out_cf_fois_ppm, cmap='gray')
    plt.show() """
    
    if path_pred_cf is not None and path_pred_ppm_man is not None and path_pred_ppm_cf is not None and path_pred_GT is not None and path_pred_man is not None and path_pred_cf_ppm25 is not None and path_pred_cf_ppm50 is not None and path_pred_cf_ppm75 is not None and path_pred_cf_ppm1 is not None and path_pred_cf_sur_ppm is not None:
        count_visited_cf = len(visited_cf)
        count_visited_ppm_man = len(visited_ppm_man)
        count_visited_ppm_cf = len(visited_ppm_cf)
        count_visited_GT = len(path_pred_GT)
        count_visited_man = len(visited_man)
        count_visited_cf_ppm25 = len(visited_cf_ppm25)
        count_visited_cf_ppm50 = len(visited_cf_ppm50)
        count_visited_cf_ppm75 = len(visited_cf_ppm75)
        count_visited_cf_ppm1 = len(visited_cf_ppm1)
        count_visited_cf_sur_ppm = len(visited_cf_sur_ppm)
        
        return len(path_pred_cf), len(path_pred_ppm_man), len(path_pred_ppm_cf), len(path_pred_GT), len(path_pred_man), len(path_pred_cf_ppm25), len(path_pred_cf_ppm50), len(path_pred_cf_ppm75), len(path_pred_cf_ppm1), len(path_pred_cf_sur_ppm), count_visited_cf, count_visited_ppm_man, count_visited_ppm_cf, count_visited_GT, count_visited_man, count_visited_cf_ppm25, count_visited_cf_ppm50, count_visited_cf_ppm75, count_visited_cf_ppm1, count_visited_cf_sur_ppm, rt_cf, rt_ppm_man, rt_ppm_cf, rt_man, rt_cf_ppm_25, rt_cf_ppm_50, rt_cf_ppm_75, rt_cf_ppm_1, rt_cf_sur_ppm
    return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

# MP and Tiled-MP Models

In [8]:
# Evaluate on 64x64 images
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv2DBlock_ppm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv2DBlock_ppm, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResNetBlock_ppm(nn.Module):
    def __init__(self, channels):
        super(ResNetBlock_ppm, self).__init__()
        self.block = nn.Sequential(
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        )
        
    def forward(self, x):
        return x + self.block(x)

class DownsampleBlock_ppm(nn.Module):
    def __init__(self, channels):
        super(DownsampleBlock_ppm, self).__init__()
        self.downsample = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x):
        return self.downsample(x)

class UpsampleBlock_ppm(nn.Module):
    def __init__(self, channels):
        super(UpsampleBlock_ppm, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels, channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        
    def forward(self, x):
        return self.upsample(x)

class TransformerBlock_ppm(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(TransformerBlock_ppm, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(inplace=True),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)
        
    def forward(self, x):
        #x = x.permute(2, 0, 1)  # (B, H*W, C) -> (H*W, B, C)
        x_cal = self.norm1(x)
        # multi head selfattention with 4 heads
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        #do it again
        x_cal = self.norm1(x)
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        # layer normalization
        x_cal = self.norm2(x)
        # feed forward network
        x_cal = self.ff(x_cal)
        return x + x_cal

class Encoder_ppm(nn.Module):
    def __init__(self):
        super(Encoder_ppm, self).__init__()
        self.initial_conv = Conv2DBlock_ppm(3, 64, kernel_size=5, stride=1, padding=2)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_ppm(64),
                DownsampleBlock_ppm(64)
            ) for _ in range(3)
        ])

    def forward(self, x):
        x = self.initial_conv(x)
        for block in self.blocks:
            x = block(x)
        b, c, h, w = x.size()
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, c, x.device)
        
        x = x.flatten(2).permute(2, 0, 1)  # (B, C, H, W) -> (H*W, B, C)
        x = x + positional_embeddings
        return x, h, w
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(height * width, 1, channels, device=device)
        return pos_embed

class Decoder_ppm(nn.Module):
    def __init__(self):
        super(Decoder_ppm, self).__init__()
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_ppm(64),
                UpsampleBlock_ppm(64)
            ) for _ in range(3)
        ])
        self.final_conv = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x, h, w):
        x = x.permute(1, 2, 0).view(-1, 64, h, w)  # (H*W, B, C) -> (B, C, H, W)
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, 64, x.device)
        
        x = x + positional_embeddings
        for block in self.blocks:
            x = block(x)
        x = self.tanh(self.final_conv(x))
        return x
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(1, channels, height, width, device=device)
        return pos_embed

class PPMModel_ppm(nn.Module):
    def __init__(self):
        super(PPMModel_ppm, self).__init__()
        self.encoder = Encoder_ppm()
        self.transformer_blocks = nn.ModuleList([TransformerBlock_ppm(64, 8) for _ in range(4)])
        self.decoder = Decoder_ppm()
        
    def forward(self, x):
        x, h, w = self.encoder(x)
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = self.decoder(x, h, w)
        return x



class Conv2DBlock_cf(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv2DBlock_cf, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResNetBlock_cf(nn.Module):
    def __init__(self, channels):
        super(ResNetBlock_cf, self).__init__()
        self.block = nn.Sequential(
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        )
        
    def forward(self, x):
        return x + self.block(x)

class DownsampleBlock_cf(nn.Module):
    def __init__(self, channels):
        super(DownsampleBlock_cf, self).__init__()
        self.downsample = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x):
        return self.downsample(x)

class UpsampleBlock_cf(nn.Module):
    def __init__(self, channels):
        super(UpsampleBlock_cf, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels, channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        
    def forward(self, x):
        return self.upsample(x)

class TransformerBlock_cf(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(TransformerBlock_cf, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(inplace=True),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)
        
    def forward(self, x):
        #x = x.permute(2, 0, 1)  # (B, H*W, C) -> (H*W, B, C)
        x_cal = self.norm1(x)
        # multi head selfattention with 4 heads
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        #do it again
        x_cal = self.norm1(x)
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        # layer normalization
        x_cal = self.norm2(x)
        # feed forward network
        x_cal = self.ff(x_cal)
        return x + x_cal

class Encoder_cf(nn.Module):
    def __init__(self):
        super(Encoder_cf, self).__init__()
        self.initial_conv = Conv2DBlock_cf(3, 64, kernel_size=5, stride=1, padding=2)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_cf(64),
                DownsampleBlock_cf(64)
            ) for _ in range(3)
        ])

    def forward(self, x):
        x = self.initial_conv(x)
        for block in self.blocks:
            x = block(x)
        b, c, h, w = x.size()
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, c, x.device)
        
        x = x.flatten(2).permute(2, 0, 1)  # (B, C, H, W) -> (H*W, B, C)
        x = x + positional_embeddings
        return x, h, w
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(height * width, 1, channels, device=device)
        return pos_embed

class Decoder_cf(nn.Module):
    def __init__(self):
        super(Decoder_cf, self).__init__()
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_cf(64),
                UpsampleBlock_cf(64)
            ) for _ in range(3)
        ])
        self.final_conv = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x, h, w):
        x = x.permute(1, 2, 0).view(-1, 64, h, w)  # (H*W, B, C) -> (B, C, H, W)
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, 64, x.device)
        
        x = x + positional_embeddings
        for block in self.blocks:
            x = block(x)
        x = self.tanh(self.final_conv(x))
        return x
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(1, channels, height, width, device=device)
        return pos_embed

class PPMModel_cf(nn.Module):
    def __init__(self):
        super(PPMModel_cf, self).__init__()
        self.encoder = Encoder_cf()
        self.transformer_blocks = nn.ModuleList([TransformerBlock_cf(64, 8) for _ in range(4)])
        self.decoder = Decoder_cf()
        
    def forward(self, x):
        x, h, w = self.encoder(x)
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = self.decoder(x, h, w)
        return x

# Evaluate on 64x64 images
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv2DBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv2DBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResNetBlock(nn.Module):
    def __init__(self, channels):
        super(ResNetBlock, self).__init__()
        self.block = nn.Sequential(
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        )
        
    def forward(self, x):
        return x + self.block(x)

class DownsampleBlock(nn.Module):
    def __init__(self, channels):
        super(DownsampleBlock, self).__init__()
        self.downsample = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x):
        return self.downsample(x)

class UpsampleBlock(nn.Module):
    def __init__(self, channels):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels, channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        
    def forward(self, x):
        return self.upsample(x)

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(inplace=True),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)
        
    def forward(self, x):
        #x = x.permute(2, 0, 1)  # (B, H*W, C) -> (H*W, B, C)
        x_cal = self.norm1(x)
        # multi head selfattention with 4 heads
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        #do it again
        x_cal = self.norm1(x)
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        # layer normalization
        x_cal = self.norm2(x)
        # feed forward network
        x_cal = self.ff(x_cal)
        return x + x_cal

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.initial_conv = Conv2DBlock(3, 64, kernel_size=5, stride=1, padding=2)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock(64),
                DownsampleBlock(64)
            ) for _ in range(3)
        ])

    def forward(self, x):
        x = self.initial_conv(x)
        for block in self.blocks:
            x = block(x)
        b, c, h, w = x.size()
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, c, x.device)
        
        x = x.flatten(2).permute(2, 0, 1)  # (B, C, H, W) -> (H*W, B, C)
        x = x + positional_embeddings
        return x, h, w
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(height * width, 1, channels, device=device)
        return pos_embed

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock(64),
                UpsampleBlock(64)
            ) for _ in range(3)
        ])
        self.final_conv = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x, h, w):
        x = x.permute(1, 2, 0).view(-1, 64, h, w)  # (H*W, B, C) -> (B, C, H, W)
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, 64, x.device)
        
        x = x + positional_embeddings
        for block in self.blocks:
            x = block(x)
        x = self.tanh(self.final_conv(x))
        return x
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(1, channels, height, width, device=device)
        return pos_embed

class PPMModel(nn.Module):
    def __init__(self):
        super(PPMModel, self).__init__()
        self.encoder = Encoder()
        self.transformer_blocks = nn.ModuleList([TransformerBlock(64, 8) for _ in range(4)])
        self.decoder = Decoder()
        
    def forward(self, x):
        x, h, w = self.encoder(x)
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = self.decoder(x, h, w)
        return x

In [12]:
class Conv2DBlock_cf2(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv2DBlock_cf2, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResNetBlock_cf2(nn.Module):
    def __init__(self, channels):
        super(ResNetBlock_cf2, self).__init__()
        self.block = nn.Sequential(
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(32, channels),
            nn.SiLU(),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        )
        
    def forward(self, x):
        return x + self.block(x)

class DownsampleBlock_cf2(nn.Module):
    def __init__(self, channels):
        super(DownsampleBlock_cf2, self).__init__()
        self.downsample = nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1)
        
    def forward(self, x):
        return self.downsample(x)

class UpsampleBlock_cf2(nn.Module):
    def __init__(self, channels):
        super(UpsampleBlock_cf2, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels, channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        
    def forward(self, x):
        return self.upsample(x)

class TransformerBlock_cf2(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(TransformerBlock_cf2, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(inplace=True),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)
        
    def forward(self, x):
        #x = x.permute(2, 0, 1)  # (B, H*W, C) -> (H*W, B, C)
        x_cal = self.norm1(x)
        # multi head selfattention with 4 heads
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        #do it again
        x_cal = self.norm1(x)
        x_cal = self.attention(x_cal, x_cal, x_cal)[0]
        x = x + x_cal
        # layer normalization
        x_cal = self.norm2(x)
        # feed forward network
        x_cal = self.ff(x_cal)
        return x + x_cal

class Encoder_cf2(nn.Module):
    def __init__(self):
        super(Encoder_cf2, self).__init__()
        self.initial_conv = Conv2DBlock_cf(2, 64, kernel_size=5, stride=1, padding=2)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_cf2(64),
                DownsampleBlock_cf2(64)
            ) for _ in range(3)
        ])

    def forward(self, x):
        x = self.initial_conv(x)
        for block in self.blocks:
            x = block(x)
        b, c, h, w = x.size()
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, c, x.device)
        
        x = x.flatten(2).permute(2, 0, 1)  # (B, C, H, W) -> (H*W, B, C)
        x = x + positional_embeddings
        return x, h, w
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(height * width, 1, channels, device=device)
        return pos_embed

class Decoder_cf2(nn.Module):
    def __init__(self):
        super(Decoder_cf2, self).__init__()
        self.blocks = nn.ModuleList([
            nn.Sequential(
                ResNetBlock_cf2(64),
                UpsampleBlock_cf2(64)
            ) for _ in range(3)
        ])
        self.final_conv = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, x, h, w):
        x = x.permute(1, 2, 0).view(-1, 64, h, w)  # (H*W, B, C) -> (B, C, H, W)
        
        # Create positional embeddings dynamically based on spatial dimensions
        positional_embeddings = self.get_positional_embeddings(h, w, 64, x.device)
        
        x = x + positional_embeddings
        for block in self.blocks:
            x = block(x)
        x = self.tanh(self.final_conv(x))
        return x
    
    def get_positional_embeddings(self, height, width, channels, device):
        pos_embed = torch.randn(1, channels, height, width, device=device)
        return pos_embed

class PPMModel_cf2(nn.Module):
    def __init__(self):
        super(PPMModel_cf2, self).__init__()
        self.encoder = Encoder_cf2()
        self.transformer_blocks = nn.ModuleList([TransformerBlock_cf2(64, 8) for _ in range(4)])
        self.decoder = Decoder_cf2()
        
    def forward(self, x):
        x, h, w = self.encoder(x)
        for transformer in self.transformer_blocks:
            x = transformer(x)
        x = self.decoder(x, h, w)
        return x

# Evaluation Procedure

In [20]:
# do a statistical analysis on the test dataset
import pandas as pd
import time
import tqdm
import numpy as np
from PIL import Image

model_ppm = PPMModel_ppm()
#model_ppm.load_state_dict(torch.load("resized_ppm.pth")['state_dict'])

model_cf = PPMModel_cf()
#model_cf.load_state_dict(torch.load("resized_cf.pth")['state_dict'])

#model_ppm_cf = PPMModel()
model_ppm.load_state_dict(torch.load("new_model_aug_ppm.pth"))
model_cf.load_state_dict(torch.load("new_model_aug_cf.pth"))

#ici

model_ppm.eval()
model_cf.eval()
#model_ppm_cf.eval()

results = []
optimal_found_ppm_man = 0
optimal_found_ppm_cf = 0
optimal_found_cf = 0
optimal_found_cf_ppm25 = 0
optimal_found_cf_ppm50 = 0
optimal_found_cf_ppm75 = 0
optimal_found_cf_ppm1 = 0
optimal_found_cf_sur_ppm = 0
no_path = 0

instance_complexity = 1

test = create_dataset("../32x MP_dataset_unclassed_augmented_cf/test")

x_test = []
y_test = []

for i in range(len(test)):
    x_test.append((test[i][0], test[i][1]))
    y_test.append(test[i][2])

# input3 is a tensor of 0
#input3 = torch.zeros(1, 1, 32, 32)

# n_steps = size of the test dataset
n_steps = len(x_test)
for i in tqdm.tqdm(range(n_steps)):
    with torch.no_grad():
        x_batch, y_batch = get_non_random_batch(x_test, y_test, 1, i)[0]
        type_of_map = x_batch[0].split('/')[-1].split('__')[0]
        input1 = torch.tensor(np.array([np.array(Image.open(x_batch[0])) / 255]), dtype=torch.float32).unsqueeze(1)
        goal = torch.tensor(np.array([np.array(Image.open(x_batch[1])) / 255]), dtype=torch.float32).unsqueeze(1)
        goal_pos = np.where(goal[0][0] == 1)
        goal_pos = (goal_pos[0][0], goal_pos[1][0])
        
        for _ in range(3):
            start = goal_pos
            while manhattan_distance(start, goal_pos) < 30 or input1[0][0][start[0], start[1]] == 0:
                start = (np.random.randint(0, 32), np.random.randint(0, 32))
            
            # start_goal is goal image and we add the start position to it
            image_goal = np.array(Image.open(x_batch[1])) / 255
            # add start
            image_goal[start[0], start[1]] = 1
            start_goal = torch.tensor(np.array([image_goal]), dtype=torch.float32).unsqueeze(1)
            
            input_cf = torch.cat((input1, goal), dim=1)
            
            input_ppm = torch.cat((input1, start_goal), dim=1)
            
            rt_inf_ppm = time.time()
            output_ppm = model_ppm(input_ppm)
            rt_inf_ppm = time.time() - rt_inf_ppm
            output_ppm = output_ppm.detach().squeeze().numpy()
            
            rt_inf_cf = time.time()
            output_cf = model_cf(input_cf)
            rt_inf_cf = time.time() - rt_inf_cf
            output_cf = output_cf.detach().squeeze().numpy()
            
            path_cf, path_ppm_man, path_ppm_cf, path_GT, path_man, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, vis_cf, vis_ppm_man, vis_ppm_cf, vis_GT, vis_man, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, rt_cf, rt_ppm_man, rt_ppm_cf, rt_man, rt_cf_ppm_25, rt_cf_ppm_50, rt_cf_ppm_75, rt_cf_ppm_1, rt_cf_sur_ppm = everything_new(2, input1[0][0], start, goal_pos, output_ppm, output_cf, 1.25)
            
            if path_cf is None or path_ppm_man is None or path_ppm_cf is None or path_GT is None or path_man is None or path_cf_ppm25 is None or path_cf_ppm50 is None or path_cf_ppm75 is None or path_cf_ppm1 is None or path_cf_sur_ppm is None:
                no_path += 1
                continue
            
            #print(path_pred_cf, path_pred_ppm, path_pred_ppm_cf, path_pred_ppm_x_cf, path_GT, path_manhattan, visited_pred_cf, visited_pred_ppm, visited_pred_ppm_cf, visited_pred_ppm_x_cf, visited_GT, visited_manhattan, rt_cf, rt_ppm, rt_ppm_cf, rt_ppm_x_cf, rt_manhattan)
            
            optimal_found_ppm_man = 0
            optimal_found_ppm_cf = 0
            optimal_found_cf = 0
            optimal_found_cf_ppm25 = 0
            optimal_found_cf_ppm50 = 0
            optimal_found_cf_ppm75 = 0
            optimal_found_cf_ppm1 = 0
            optimal_found_cf_sur_ppm = 0
            # instance complexity is defined by cost(π∗(start, goal))/h(start)
            instance_complexity = path_GT / manhattan_distance(start, goal_pos)
            
            if path_GT == path_cf:
                optimal_found_cf = 1
            if path_GT == path_ppm_man:
                optimal_found_ppm_man = 1
            if path_GT == path_ppm_cf:
                optimal_found_ppm_cf = 1
            if path_GT == path_cf_ppm25:
                optimal_found_cf_ppm25 = 1
            if path_GT == path_cf_ppm50:
                optimal_found_cf_ppm50 = 1
            if path_GT == path_cf_ppm75:
                optimal_found_cf_ppm75 = 1
            if path_GT == path_cf_ppm1:
                optimal_found_cf_ppm1 = 1
            if path_GT == path_cf_sur_ppm:
                optimal_found_cf_sur_ppm = 1
                
            results.append([type_of_map, instance_complexity, 100*optimal_found_cf, 100*optimal_found_ppm_man, 100*optimal_found_ppm_cf, 100*optimal_found_cf_ppm25, 100*optimal_found_cf_ppm50, 100*optimal_found_cf_ppm75, 100*optimal_found_cf_ppm1, 100*optimal_found_cf_sur_ppm, path_cf, path_ppm_man, path_ppm_cf, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, path_GT, path_man, vis_cf, vis_ppm_man, vis_ppm_cf, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, vis_GT, vis_man, 1000*rt_cf, 1000*rt_ppm_man, 1000*rt_ppm_cf, 1000*rt_man, 1000*rt_cf_ppm_25, 1000*rt_cf_ppm_50, 1000*rt_cf_ppm_75, 1000*rt_cf_ppm_1, 1000*rt_cf_sur_ppm, 1000*rt_inf_cf, 1000*rt_inf_ppm])

results = pd.DataFrame(results, columns=['Type of map', 'Instance Complexity', 'Optimal Found CF', 'Optimal Found PPM+MAN', 'Optimal Found PPM+CF', 'Optimal Found CF+0.25*PPM', 'Optimal Found CF+0.50*PPM', 'Optimal Found CF+0.75*PPM', 'Optimal Found CF+1*PPM', 'Optimal Found CF/PPM', 'Predicted Path Length CF', 'Predicted Path Length PPM+MAN', 'Predicted Path Length PPM+CF', 'Predicted Path Length CF+0.25*PPM', 'Predicted Path Length CF+0.50*PPM', 'Predicted Path Length CF+0.75*PPM', 'Predicted Path Length CF+1*PPM', 'Predicted Path Length CF/PPM', 'Ground Truth Path Length', 'Manhattan Path Length', 'Expansion CF', 'Expansion PPM+MAN', 'Expansion PPM+CF', 'Expansion CF+0.25*PPM', 'Expansion CF+0.50*PPM', 'Expansion CF+0.75*PPM', 'Expansion CF+1*PPM', 'Expansion CF/PPM', 'Expansion GT', 'Expansion Manhattan', 'Run Time CF', 'Run Time PPM+MAN', 'Run Time PPM+CF', 'Run Time Manhattan', 'Run Time CF+0.25*PPM', 'Run Time CF+0.50*PPM', 'Run Time CF+0.75*PPM', 'Run Time CF+1*PPM', 'Run Time CF/PPM', 'Run Time Inference CF', 'Run Time Inference PPM'])

print("No path found:", no_path)
print(results.describe())

# print describe all the things tha contains CF/PPM in the name of the column
#print(results.filter(like='CF/PPM').describe())
#print(results.filter(like='Exp').describe())

100%|██████████| 32000/32000 [7:11:36<00:00,  1.24it/s]  


No path found: 9696
       Instance Complexity  Optimal Found CF  Optimal Found PPM+MAN  \
count         86304.000000      86304.000000           86304.000000   
mean              1.157167         88.453606              73.150723   
std               0.254528         31.958286              44.317793   
min               1.016129          0.000000               0.000000   
25%               1.027027        100.000000               0.000000   
50%               1.031250        100.000000             100.000000   
75%               1.166667        100.000000             100.000000   
max               2.967742        100.000000             100.000000   

       Optimal Found PPM+CF  Optimal Found CF+0.25*PPM  \
count          86304.000000               86304.000000   
mean              72.234195                  93.090703   
std               44.784638                  25.361364   
min                0.000000                   0.000000   
25%                0.000000                 100.00

In [22]:
# save the results in a csv file
results.to_csv("no_w_espoir_final_32.csv", index=False)

# Tiled-MP dataset

In [9]:
import glob
import tqdm
import cv2
import numpy as np

class Dataset:
    def __init__(self, root, rem512=False, rem256=False, rem16=False, uniform=False, rem128=False):
        files = []
        dict = {}
        
        def get_base_filename(filename):
            return filename.split('_start')[0] + '.png'
        
        for file in glob.glob(root + '/*'):
            if rem512:
                if "512x512" in file:
                    continue
            if rem256:
                if "256x256" in file:
                    continue
            if rem16:
                if "16x16" in file:
                    continue
            if rem128:
                if "128x128" in file:
                    continue
            if "GT" not in file and "start" not in file:
                base_filename = file.split(".")[0]
                for i in range(2):
                    files.append(base_filename + "_" + str(i) + ".png")
            elif "start" in file:
                base_filename = get_base_filename(file)
                dict[base_filename] = file
        
        if uniform:
            files_16, files_32, files_64, files_96, files_128, files_256, files_512 = [], [], [], [], [], [], []
            for file in files:
                if "16x16" in file:
                    files_16.append(file)
                elif "32x32" in file:
                    files_32.append(file)
                elif "64x64" in file:
                    files_64.append(file)
                elif "96x96" in file:
                    files_96.append(file)
                elif "128x128" in file:
                    files_128.append(file)
                elif "256x256" in file:
                    files_256.append(file)
                elif "512x512" in file:
                    files_512.append(file)
                
            min = 200 # TODO: change 
            for files in [files_16, files_32, files_64, files_96, files_128, files_256, files_512]:
                if len(files) < min and len(files) > 0:
                    min = len(files)
            if len(files_16) > 0:
                files_16 = np.random.choice(files_16, min, replace=False)
            if len(files_32) > 0:
                files_32 = np.random.choice(files_32, min, replace=False)
            if len(files_64) > 0:
                files_64 = np.random.choice(files_64, min, replace=False)
            if len(files_96) > 0:
                files_96 = np.random.choice(files_96, min, replace=False)
            if len(files_128) > 0:
                files_128 = np.random.choice(files_128, min, replace=False)
            if len(files_256) > 0:
                files_256 = np.random.choice(files_256, min, replace=False)
            if len(files_512) > 0:
                files_512 = np.random.choice(files_512, min, replace=False)
            files = list(files_16) + list(files_32) + list(files_64) + list(files_96) + list(files_128) + list(files_256) + list(files_512)
        
        self.inputs1 = []
        self.inputs2 = []
        self.targets = []
        
        for file in tqdm.tqdm(files):
            name = file.split('.')[0]
            self.inputs1.append(name[:-2] + ".png")
            self.inputs2.append(dict[file])
            self.targets.append([name + "_GT_cf.png", name + "_GT_ppm.png"])
    
    def __len__(self):
        return len(self.inputs1)
    
    def __getitem__(self, idx):
        input1 = cv2.imread(self.inputs1[idx], cv2.IMREAD_GRAYSCALE)
        input2 = cv2.imread(self.inputs2[idx], cv2.IMREAD_GRAYSCALE)
        targets = [cv2.imread(target, cv2.IMREAD_GRAYSCALE) for target in self.targets[idx]]
        
        goal = self.inputs2[idx].split('_end-')[1].split('.png')[0].split('_')
        goal = (int(goal[0]), int(goal[1]))
        # create an image the same size as input1 with the goal point
        input3 = np.zeros(input1.shape, dtype=np.float32)
        input3[goal[0], goal[1]] = 1
        input3 = input3.astype('float32')
        
        input1 = input1.astype('float32')
        input2 = input2.astype('float32')
        targets = [target.astype('float32') for target in targets]
        
        return input1, input2, input3, targets

In [10]:
resized_test_dataset = Dataset("C:/Users/Elie/Desktop/Individual Research Project/Data/MP dataset/32x MP_dataset_unclassed_resized_all/test", rem512=True, rem256=False, rem16=True, uniform=True, rem128=False)

100%|██████████| 1000/1000 [00:00<00:00, 200953.62it/s]


In [11]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image

class Loader_non_resize:
    def __init__(self, dataset, batch_size, device):
        self.device = device
        split_indices = list(range(len(dataset)))
        sampler = torch.utils.data.sampler.SubsetRandomSampler(split_indices)
        self.loader = DataLoader(dataset, batch_size=batch_size, sampler=sampler, num_workers=0, collate_fn=self.collate_fn)
    
    def __len__(self):
        return len(self.loader)
    
    def __iter__(self):
        for data in self.loader:
            yield [d.to(self.device) for d in data]
    
    def collate_fn(self, batch):
        images1 = [Image.fromarray((item[0]).astype('uint8')) for item in batch]
        images2 = [Image.fromarray((item[1]).astype('uint8')) for item in batch]
        images3 = [Image.fromarray((item[2]).astype('uint8')) for item in batch]
        targets = [Image.fromarray((target).astype('uint8')) for item in batch for target in item[3]]
        
        # to tensor
        transform = transforms.ToTensor()
        images1 = [transform(image) for image in images1]
        images2 = [transform(image) for image in images2]
        images3 = [transform(image) for image in images3]
        targets = [transform(image) for image in targets]

        # return torch stack
        return torch.stack(images1), torch.stack(images2), torch.stack(images3), torch.stack(targets)

# Evaluation Procedure on Tiled-MP dataset

In [14]:
# do a statistical analysis on the test dataset
import pandas as pd
import time
import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch

model_ppm = PPMModel_ppm()
model_cf = PPMModel_cf()
#model_ppm_cf = PPMModel()

#model_ppm.load_state_dict(torch.load("new_model_aug_ppm.pth"))
model_ppm.load_state_dict(torch.load("resized_ppm.pth")["state_dict"])

model_cf.load_state_dict(torch.load("resized_cf.pth")["state_dict"])

#model_ppm_cf.load_state_dict(torch.load("new_model_aug_both.pth"))

model_ppm.to('cuda')
model_cf.to('cuda')
#model_ppm_cf.to('cuda')

model_ppm.eval()
model_cf.eval()
#model_ppm_cf.eval()

# create Loader
loader = Loader_non_resize(resized_test_dataset, 1, 'cuda')

"""
# take 1 using next
input1, input2, input3, targets = next(iter(loader))

# plot everything
fig, ax = plt.subplots(1, 6, figsize=(15, 5))
ax[0].imshow(input1[0][0].cpu().detach().numpy(), cmap='gray')
ax[0].set_title('Input 1')
ax[1].imshow(input2[0][0].cpu().detach().numpy(), cmap='gray')
ax[1].set_title('Input 2')
ax[2].imshow(input3[0][0].cpu().detach().numpy(), cmap='gray')
ax[2].set_title('Input 3')
ax[3].imshow(targets[0][0].cpu().detach().numpy(), cmap='gray')
ax[3].set_title('Target CF')
ax[4].imshow(targets[1][0].cpu().detach().numpy(), cmap='gray')
ax[4].set_title('Target PPM')
ax[5].imshow(targets[2][0].cpu().detach().numpy(), cmap='gray')
ax[5].set_title('Target PPM+CF')
plt.show()

input1, input2, input3, targets = input1.to('cuda'), input2.to('cuda'), input3.to('cuda'), targets.to('cuda')

input_cf = torch.cat((input1, input3), dim=1)
input_ppm = torch.cat((input1, input2), dim=1)
input_ppm_cf = torch.cat((input1, input2, input3), dim=1)

output_cf = model_cf(input_cf)
output_ppm = model_ppm(input_ppm)
output_ppm_cf = model_ppm_cf(input_ppm_cf)

# plot everything
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(output_cf[0][0].cpu().detach().numpy(), cmap='gray')
ax[0].set_title('Output CF')
ax[1].imshow(output_ppm[0][0].cpu().detach().numpy(), cmap='gray')
ax[1].set_title('Output PPM')
ax[2].imshow(output_ppm_cf[0][0].cpu().detach().numpy(), cmap='gray')
ax[2].set_title('Output PPM+CF')
plt.show() 
"""

# ici

results = []
optimal_found_cf = 0
optimal_found_ppm_man = 0
optimal_found_ppm_cf = 0
optimal_found_cf_ppm25 = 0
optimal_found_cf_ppm50 = 0
optimal_found_cf_ppm75 = 0
optimal_found_cf_ppm1 = 0
optimal_found_cf_sur_ppm = 0
no_path = 0

instance_complexity = 1

for input1, input2, input3, target in tqdm.tqdm(loader):
    with torch.no_grad():
        size = str(input1[0][0].size()[0])
        # create input4: tensor of 0s with the same size as input1
        input4 = torch.zeros_like(input1)
        input_cf = torch.cat((input1, input3, input4), dim=1) #, input4), dim=1)
        input_ppm = torch.cat((input1, input2, input4), dim=1) #, input4), dim=1)
        #input_ppm_cf = torch.cat((input1, input2, input3), dim=1)
        start_time_inference_cf = time.time()
        output_cf = model_cf(input_cf)
        rt_inference_cf = time.time() - start_time_inference_cf
        start_time_inference_ppm = time.time()
        output_ppm = model_ppm(input_ppm)
        rt_inference_ppm = time.time() - start_time_inference_ppm
        #start_time_inference_ppm_cf = time.time()
        #output_ppm_cf = model_ppm_cf(input_ppm_cf)
        #rt_inference_ppm_cf = time.time() - start_time_inference_ppm_cf
        
        input2 = input2.detach().cpu().numpy()
        input3 = input3.detach().cpu().numpy() * 255
        goal = np.where(input3[0][0] == 1)
        goal_pos = (goal[0][0], goal[1][0])
        # start is the point where input 3 is 1 that is not the goal
        start = np.where(input2[0][0] != 0)
        if start[0][0] == goal_pos[0] and start[1][0] == goal_pos[1]:
            start = (start[0][1], start[1][1])
        else:
            start = (start[0][0], start[1][0])
        
        #rt_inference_ppm_x_cf = rt_inference_ppm + rt_inference_cf
        #start_time_inference_ppm_x_cf = time.time()
        #output_ppm_x_cf = output_ppm * output_cf
        #rt_inference_ppm_x_cf = time.time() - start_time_inference_ppm_x_cf + rt_inference_ppm_x_cf
        
        #output_ppm_x_cf = output_ppm_x_cf.detach().cpu().numpy().squeeze()
        
        # detach cpu unsqueeze and numpy the outputs
        output_ppm = output_ppm.detach().cpu().numpy().squeeze()
        output_cf = output_cf.detach().cpu().numpy().squeeze()
        #output_ppm_cf = output_ppm_cf.detach().cpu().numpy().squeeze()
        
        optimal_found_cf = 0
        optimal_found_ppm_man = 0
        optimal_found_ppm_cf = 0
        optimal_found_cf_ppm25 = 0
        optimal_found_cf_ppm50 = 0
        optimal_found_cf_ppm75 = 0
        optimal_found_cf_ppm1 = 0
        optimal_found_cf_sur_ppm = 0
        
        # use everything_new function
        path_cf, path_ppm_man, path_ppm_cf, path_GT, path_man, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, vis_cf, vis_ppm_man, vis_ppm_cf, vis_GT, vis_man, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, rt_cf, rt_ppm_man, rt_ppm_cf, rt_man, rt_cf_ppm_25, rt_cf_ppm_50, rt_cf_ppm_75, rt_cf_ppm_1, rt_cf_sur_ppm = everything_new(2, input1[0][0], start, goal_pos, output_ppm, output_cf, 1.25)
        
        if path_GT == None:
            no_path += 1
            continue
        
        # instance complexity is defined by cost(π∗(start, goal))/h(start)
        instance_complexity = path_GT / manhattan_distance(start, goal_pos)
        
        if path_cf == path_GT:
            optimal_found_cf = 1
        if path_ppm_man == path_GT:
            optimal_found_ppm_man = 1
        if path_ppm_cf == path_GT:
            optimal_found_ppm_cf = 1
        if path_cf_ppm25 == path_GT:
            optimal_found_cf_ppm25 = 1
        if path_cf_ppm50 == path_GT:
            optimal_found_cf_ppm50 = 1
        if path_cf_ppm75 == path_GT:
            optimal_found_cf_ppm75 = 1
        if path_cf_ppm1 == path_GT:
            optimal_found_cf_ppm1 = 1
        if path_cf_sur_ppm == path_GT:
            optimal_found_cf_sur_ppm = 1
        
        results.append([size, instance_complexity, 100*optimal_found_cf, 100*optimal_found_ppm_man, 100*optimal_found_ppm_cf, 100*optimal_found_cf_ppm25, 100*optimal_found_cf_ppm50, 100*optimal_found_cf_ppm75, 100*optimal_found_cf_ppm1, 100*optimal_found_cf_sur_ppm, path_cf, path_ppm_man, path_ppm_cf, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, path_GT, path_man, vis_cf, vis_ppm_man, vis_ppm_cf, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, vis_GT, vis_man, 1000*rt_cf, 1000*rt_ppm_man, 1000*rt_ppm_cf, 1000*rt_man, 1000*rt_cf_ppm_25, 1000*rt_cf_ppm_50, 1000*rt_cf_ppm_75, 1000*rt_cf_ppm_1, 1000*rt_cf_sur_ppm, 1000*rt_inference_cf, 1000*rt_inference_ppm])

results = pd.DataFrame(results, columns=['Size', 'Instance Complexity', 'Optimal Found CF', 'Optimal Found PPM+MAN', 'Optimal Found PPM+CF', 'Optimal Found CF+0.25*PPM', 'Optimal Found CF+0.50*PPM', 'Optimal Found CF+0.75*PPM', 'Optimal Found CF+1*PPM', 'Optimal Found CF/PPM', 'Predicted Path Length CF', 'Predicted Path Length PPM+MAN', 'Predicted Path Length PPM+CF', 'Predicted Path Length CF+0.25*PPM', 'Predicted Path Length CF+0.50*PPM', 'Predicted Path Length CF+0.75*PPM', 'Predicted Path Length CF+1*PPM', 'Predicted Path Length CF/PPM', 'Ground Truth Path Length', 'Manhattan Path Length', 'Expansion CF', 'Expansion PPM+MAN', 'Expansion PPM+CF', 'Expansion CF+0.25*PPM', 'Expansion CF+0.50*PPM', 'Expansion CF+0.75*PPM', 'Expansion CF+1*PPM', 'Expansion CF/PPM', 'Expansion GT', 'Expansion Manhattan', 'Run Time CF', 'Run Time PPM+MAN', 'Run Time PPM+CF', 'Run Time Manhattan', 'Run Time CF+0.25*PPM', 'Run Time CF+0.50*PPM', 'Run Time CF+0.75*PPM', 'Run Time CF+1*PPM', 'Run Time CF/PPM', 'Run Time Inference CF', 'Run Time Inference PPM'])

print("No path found:", no_path)
print(results.describe())

# save 
results.to_csv("no_w_resized_espoir_results_on_new_dataset.csv", index=False) 

100%|██████████| 1000/1000 [6:45:17<00:00, 24.32s/it]   

No path found: 39
       Instance Complexity  Optimal Found CF  Optimal Found PPM+MAN  \
count           961.000000        961.000000             961.000000   
mean              1.103907         22.164412              46.409990   
std               0.190588         41.556914              49.896919   
min               1.002336          0.000000               0.000000   
25%               1.010870          0.000000               0.000000   
50%               1.027778          0.000000               0.000000   
75%               1.103448          0.000000             100.000000   
max               2.750000        100.000000             100.000000   

       Optimal Found PPM+CF  Optimal Found CF+0.25*PPM  \
count            961.000000                 961.000000   
mean              42.559834                  47.970864   
std               49.469086                  49.984823   
min                0.000000                   0.000000   
25%                0.000000                   0.0000

# Evaluation Procedure on out-of-distribution dataset

In [17]:
path = "C:/Users/Elie/Desktop/Individual Research Project/Data/out-of-distrib_dataset"
folders0 = ["non_resized"] #, "resized256", "non_resized"]
folders1 = ["bgmaps", "da2", "maze", "street"]
import os 
import time
import tqdm
import numpy as np
import random
from PIL import Image
# import transforms
from torchvision import transforms
import torch


model_cf = PPMModel_cf()
#model_cf.load_state_dict(torch.load("new_model_aug_cf.pth"))
model_cf.load_state_dict(torch.load("resized_cf.pth")["state_dict"])
#new_model_aug_cf.pth

model_ppm = PPMModel_ppm()
#model_ppm.load_state_dict(torch.load("new_model_aug_ppm.pth"))
model_ppm.load_state_dict(torch.load("resized_ppm.pth")["state_dict"])

#model_ppm_cf = PPMModel()
#model_ppm_cf.load_state_dict(torch.load("new_model_aug_both.pth"))

model_cf.eval()
model_ppm.eval()
#model_ppm_cf.eval()

model_cf, model_ppm = model_cf.to("cpu"), model_ppm.to("cpu")

# take 1 random image in each folders1 or each folders0 of the path
images = []
fromim = {}
for folder0 in folders0:
    for folder1 in folders1:
        temp_path = path + "/" + folder0 + "/" + folder1
        files = os.listdir(temp_path)
        for file in files:
            images.append(temp_path + "/" + file)
            fromim[temp_path + "/" + file] = folder1

device = 'cpu'

no_path = 0

results = []

instance_complexity = 1

# for every image
for image in tqdm.tqdm(images):
    # load the image and get its size
    img = Image.open(image).convert("L")
    name = image.split("/")[-1]
    fromimm = fromim[image]
    size = img.size
    
    if size[0] > 300 or size[1] > 300:
        # resize the image
        img = img.resize((300, 300))
        #as gray scale
        img = img.convert("L")
        size = img.size
        for i in range(size[0]):
            for j in range(size[1]):
                if img.getpixel((i, j)) < 128 and img.getpixel((i, j)) > 0:
                    img.putpixel((i, j), 0) if np.random.rand() < 0.9 else 255
                elif img.getpixel((i, j)) > 128 and img.getpixel((i, j)) < 255:
                    img.putpixel((i, j), 255) if np.random.rand() < 0.9 else 0
        img = img.convert("L")
    
    # img to numpy
    img_arr = np.array(img)
    
    # random goal point
    goal = (random.randint(0, size[1] - 1), random.randint(0, size[0] - 1))
    while img_arr[goal[0], goal[1]] == 0:
        goal = (random.randint(0, size[1] - 1), random.randint(0, size[0] - 1))
    
    # same for start point
    start = (random.randint(0, size[1] - 1), random.randint(0, size[0] - 1))
    while img_arr[start[0], start[1]] == 0 or manhattan_distance(start, goal) < min(size[0], size[1]) / 4:
        start = (random.randint(0, size[1] - 1), random.randint(0, size[0] - 1))
    
    size = str(size[0]) + "x" + str(size[1])
    
    img = transforms.ToTensor()(img).unsqueeze(0).float()
    goal_image = torch.zeros_like(img)
    goal_image[0, 0, goal[0], goal[1]] = 1
    goal_image = goal_image.float()
    
    start_goal = torch.zeros_like(img)
    start_goal[0, 0, start[0], start[1]] = 1
    start_goal[0, 0, goal[0], goal[1]] = 1
    start_goal = start_goal.float()
    
    # to device
    img = img.to(device)
    goal_image = goal_image.to(device)
    start_goal = start_goal.to(device)
    
    # create tensor of 0s with the same size as img
    new_input = torch.zeros_like(img)
    
    input_cf = torch.cat((img, goal_image, new_input), dim=1)
    input_ppm = torch.cat((img, start_goal, new_input), dim=1)
    #input_ppm_cf = torch.cat((img, start_goal, goal_image), dim=1)
    
    rt_inf_cf = time.time()
    output_cf = model_cf(input_cf)
    rt_inf_cf = time.time() - rt_inf_cf
    rt_inf_ppm = time.time()
    output_ppm = model_ppm(input_ppm)
    rt_inf_ppm = time.time() - rt_inf_ppm
    #output_ppm_cf = model_ppm_cf(input_ppm_cf)
    
    output_cf = output_cf.detach().cpu().numpy().squeeze()
    output_ppm = output_ppm.detach().cpu().numpy().squeeze()
    
    #ici
    
    optimal_found_ppm_man = 0
    optimal_found_cf = 0
    optimal_found_ppm_cf = 0
    optimal_found_cf_ppm25 = 0
    optimal_found_cf_ppm50 = 0
    optimal_found_cf_ppm75 = 0
    optimal_found_cf_ppm1 = 0
    optimal_found_cf_sur_ppm = 0
    
    path_cf, path_ppm_man, path_ppm_cf, path_GT, path_man, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, vis_cf, vis_ppm_man, vis_ppm_cf, vis_GT, vis_man, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, rt_cf, rt_ppm_man, rt_ppm_cf, rt_man, rt_cf_ppm_25, rt_cf_ppm_50, rt_cf_ppm_75, rt_cf_ppm_1, rt_cf_sur_ppm = everything_new(2, img[0][0], start, goal, output_ppm, output_cf, 1.25)
    
    if path_GT == None:
        no_path += 1
        continue
    
    instance_complexity = path_GT / manhattan_distance(start, goal)

    if path_cf == path_GT:
        optimal_found_cf = 1
    if path_ppm_man == path_GT:
        optimal_found_ppm_man = 1
    if path_ppm_cf == path_GT:
        optimal_found_ppm_cf = 1
    if path_cf_ppm25 == path_GT:
        optimal_found_cf_ppm25 = 1
    if path_cf_ppm50 == path_GT:
        optimal_found_cf_ppm50 = 1
    if path_cf_ppm75 == path_GT:
        optimal_found_cf_ppm75 = 1
    if path_cf_ppm1 == path_GT:
        optimal_found_cf_ppm1 = 1
    if path_cf_sur_ppm == path_GT:
        optimal_found_cf_sur_ppm = 1
    
    results.append([fromimm, name, size, instance_complexity, 100*optimal_found_cf, 100*optimal_found_ppm_man, 100*optimal_found_ppm_cf, 100*optimal_found_cf_ppm25, 100*optimal_found_cf_ppm50, 100*optimal_found_cf_ppm75, 100*optimal_found_cf_ppm1, 100*optimal_found_cf_sur_ppm, path_cf, path_ppm_man, path_ppm_cf, path_cf_ppm25, path_cf_ppm50, path_cf_ppm75, path_cf_ppm1, path_cf_sur_ppm, path_GT, path_man, vis_cf, vis_ppm_man, vis_ppm_cf, vis_cf_ppm25, vis_cf_ppm50, vis_cf_ppm75, vis_cf_ppm1, vis_cf_sur_ppm, vis_GT, vis_man, 1000*rt_cf, 1000*rt_ppm_man, 1000*rt_ppm_cf, 1000*rt_man, 1000*rt_cf_ppm_25, 1000*rt_cf_ppm_50, 1000*rt_cf_ppm_75, 1000*rt_cf_ppm_1, 1000*rt_cf_sur_ppm, 1000*rt_inf_cf, 1000*rt_inf_ppm])

import pandas as pd

results = pd.DataFrame(results, columns=['From', 'Name', 'Size', 'Instance Complexity', 'Optimal Found CF', 'Optimal Found PPM+MAN', 'Optimal Found PPM+CF', 'Optimal Found CF+0.25*PPM', 'Optimal Found CF+0.50*PPM', 'Optimal Found CF+0.75*PPM', 'Optimal Found CF+1*PPM', 'Optimal Found CF/PPM', 'Predicted Path Length CF', 'Predicted Path Length PPM+MAN', 'Predicted Path Length PPM+CF', 'Predicted Path Length CF+0.25*PPM', 'Predicted Path Length CF+0.50*PPM', 'Predicted Path Length CF+0.75*PPM', 'Predicted Path Length CF+1*PPM', 'Predicted Path Length CF/PPM', 'Ground Truth Path Length', 'Manhattan Path Length', 'Expansion CF', 'Expansion PPM+MAN', 'Expansion PPM+CF', 'Expansion CF+0.25*PPM', 'Expansion CF+0.50*PPM', 'Expansion CF+0.75*PPM', 'Expansion CF+1*PPM', 'Expansion CF/PPM', 'Expansion GT', 'Expansion Manhattan', 'Run Time CF', 'Run Time PPM+MAN', 'Run Time PPM+CF', 'Run Time Manhattan', 'Run Time CF+0.25*PPM', 'Run Time CF+0.50*PPM', 'Run Time CF+0.75*PPM', 'Run Time CF+1*PPM', 'Run Time CF/PPM', 'Run Time Inference CF', 'Run Time Inference PPM'])

print("No path found:", no_path)
print(results.describe())

# save 
results.to_csv("no_w_resized_espoir_out_of_distrib_NON_resized.csv", index=False) 

100%|██████████| 337/337 [2:50:56<00:00, 30.44s/it]   

No path found: 37
       Instance Complexity  Optimal Found CF  Optimal Found PPM+MAN  \
count           300.000000        300.000000             300.000000   
mean              1.207268         33.333333              31.333333   
std               0.356416         47.219216              46.462365   
min               1.002849          0.000000               0.000000   
25%               1.022702          0.000000               0.000000   
50%               1.071429          0.000000               0.000000   
75%               1.250349        100.000000             100.000000   
max               5.063636        100.000000             100.000000   

       Optimal Found PPM+CF  Optimal Found CF+0.25*PPM  \
count            300.000000                 300.000000   
mean              28.333333                  39.666667   
std               45.136977                  49.002309   
min                0.000000                   0.000000   
25%                0.000000                   0.0000